In this lab, you will be using the Sakila database of movie rentals.

In order to optimize our inventory, we would like to know which films will be rented next month and we are asked to create a model to predict it.

* Create a query or queries to extract the information you think may be relevant for building the prediction model. 
* It should include some film features and some rental features.

In [51]:
import mysql.connector
import pandas as pd
import numpy as np

In [52]:
from mysql.connector import connect
from getpass import getpass

In [53]:
db = connect(
host = 'localhost',
user = 'root',
passwd = getpass('Enter password:'),
database = 'sakila')

Enter password:········


In [54]:
#Query to extract information

cursor = db.cursor()
query = "select l.inventory_id,l.rental_id, a.film_id, a.title, a.release_year, a.length, i.store_id, l.rental_date, l.return_date, a.rental_duration, a.replacement_cost from rental l join inventory i on l.inventory_id = i.inventory_id join film a on i.film_id = a.film_id;"
cursor.execute(query)


In [55]:
#Read the data into a Pandas dataframe

df = pd.DataFrame(cursor.fetchall())
df

,0,1,2,3,4,5,6,7,8,9,10
0,1,4863,1,ACADEMY DINOSAUR,2006,86,1,2005-07-08 19:03:15,2005-07-11 21:29:15,6,20.99
1,1,11433,1,ACADEMY DINOSAUR,2006,86,1,2005-08-02 20:13:10,2005-08-11 21:35:10,6,20.99
2,1,14714,1,ACADEMY DINOSAUR,2006,86,1,2005-08-21 21:27:43,2005-08-30 22:26:43,6,20.99
3,1,16050,1,ACADEMY DINOSAUR,2006,86,1,2022-08-26 16:54:15,2022-08-22 16:52:00,6,20.99
4,2,972,1,ACADEMY DINOSAUR,2006,86,1,2005-05-30 20:21:07,2005-06-06 00:36:07,6,20.99
...,...,...,...,...,...,...,...,...,...,...,...
16040,4581,711,1000,ZORRO ARK,2006,50,2,2005-05-29 03:49:03,2005-05-31 08:29:03,3,18.99
16041,4581,1493,1000,ZORRO ARK,2006,50,2,2005-06-15 21:50:32,2005-06-17 01:02:32,3,18.99
16042,4581,6712,1000,ZORRO ARK,2006,50,2,2005-07-12 13:24:47,2005-07-20 09:35:47,3,18.99
16043,4581,9701,1000,ZORRO ARK,2006,50,2,2005-07-31 07:32:21,2005-08-01 05:07:21,3,18.99


In [56]:
df.columns

RangeIndex(start=0, stop=11, step=1)

In [57]:
col_names = ['inventory_id', 'rental_id', 'film_id', 'title', 'release_year', 'length',
             'store_id', 'rental_date', 'return_date', 'rental_duration', 'replacement_cost']
df.columns = col_names
df.columns

Index(['inventory_id', 'rental_id', 'film_id', 'title', 'release_year',
       'length', 'store_id', 'rental_date', 'return_date', 'rental_duration',
       'replacement_cost'],
      dtype='object')

In [58]:
df.head()

,inventory_id,rental_id,film_id,title,release_year,length,store_id,rental_date,return_date,rental_duration,replacement_cost
0,1,4863,1,ACADEMY DINOSAUR,2006,86,1,2005-07-08 19:03:15,2005-07-11 21:29:15,6,20.99
1,1,11433,1,ACADEMY DINOSAUR,2006,86,1,2005-08-02 20:13:10,2005-08-11 21:35:10,6,20.99
2,1,14714,1,ACADEMY DINOSAUR,2006,86,1,2005-08-21 21:27:43,2005-08-30 22:26:43,6,20.99
3,1,16050,1,ACADEMY DINOSAUR,2006,86,1,2022-08-26 16:54:15,2022-08-22 16:52:00,6,20.99
4,2,972,1,ACADEMY DINOSAUR,2006,86,1,2005-05-30 20:21:07,2005-06-06 00:36:07,6,20.99


In [ ]:
#Analyze extracted features and transform them. 
#You may need to encode some categorical variables, or scale numerical variables

In [59]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16045 entries, 0 to 16044
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   inventory_id      16045 non-null  int64         
 1   rental_id         16045 non-null  int64         
 2   film_id           16045 non-null  int64         
 3   title             16045 non-null  object        
 4   release_year      16045 non-null  int64         
 5   length            16045 non-null  int64         
 6   store_id          16045 non-null  int64         
 7   rental_date       16045 non-null  datetime64[ns]
 8   return_date       15862 non-null  datetime64[ns]
 9   rental_duration   16045 non-null  int64         
 10  replacement_cost  16045 non-null  object        
dtypes: datetime64[ns](2), int64(7), object(2)
memory usage: 1.3+ MB


In [60]:
#transforming replacement_cost to float (shouldn't be object)

df['replacement_cost'] = df['replacement_cost'].astype(float)

In [61]:
df.isnull().sum() #check the nans
#I assumed this 183 nans are films that are not returned yet

inventory_id          0
rental_id             0
film_id               0
title                 0
release_year          0
length                0
store_id              0
rental_date           0
return_date         183
rental_duration       0
replacement_cost      0
dtype: int64

In [62]:
df = df.dropna()

In [63]:
df.isnull().sum() #sanity check

inventory_id        0
rental_id           0
film_id             0
title               0
release_year        0
length              0
store_id            0
rental_date         0
return_date         0
rental_duration     0
replacement_cost    0
dtype: int64

In [64]:
df.describe()

,inventory_id,rental_id,film_id,release_year,length,store_id,rental_duration,replacement_cost
count,15862.000000,15862.000000,15862.000000,15862.0,15862.000000,15862.000000,15862.000000,15862.000000
mean,2292.514374,7960.164607,501.256651,2006.0,114.992750,1.506241,4.937208,20.224775
std,1321.818911,4616.764694,288.430160,0.0,40.114111,0.499977,1.401618,6.083595
min,1.000000,1.000000,1.000000,2006.0,46.000000,1.000000,3.000000,9.990000
25%,1157.000000,3968.250000,255.000000,2006.0,81.000000,1.000000,4.000000,14.990000
50%,2292.500000,7934.500000,498.000000,2006.0,114.000000,2.000000,5.000000,20.990000
75%,3432.000000,11920.750000,753.000000,2006.0,148.000000,2.000000,6.000000,25.990000
max,4581.000000,16050.000000,1000.000000,2006.0,185.000000,2.000000,7.000000,29.990000


In [65]:
df.corr()

,inventory_id,rental_id,film_id,release_year,length,store_id,rental_duration,replacement_cost
inventory_id,1.000000,0.001031,0.999959,NaN,0.054973,-0.004337,-0.054851,-0.039777
rental_id,0.001031,1.000000,0.001118,NaN,-0.001254,-0.006169,-0.005288,0.006813
film_id,0.999959,0.001118,1.000000,NaN,0.055034,-0.005071,-0.054448,-0.039176
release_year,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
length,0.054973,-0.001254,0.055034,NaN,1.000000,-0.002426,0.055037,0.026970
store_id,-0.004337,-0.006169,-0.005071,NaN,-0.002426,1.000000,0.012615,-0.021832
rental_duration,-0.054851,-0.005288,-0.054448,NaN,0.055037,0.012615,1.000000,-0.027167
replacement_cost,-0.039777,0.006813,-0.039176,NaN,0.026970,-0.021832,-0.027167,1.000000


In [66]:
df = df.drop("release_year", axis=1)
df = df.drop("rental_date", axis=1)
df = df.drop("return_date", axis=1)
# decided to drop release_year as it

In [67]:
df.head()

,inventory_id,rental_id,film_id,title,length,store_id,rental_duration,replacement_cost
0,1,4863,1,ACADEMY DINOSAUR,86,1,6,20.99
1,1,11433,1,ACADEMY DINOSAUR,86,1,6,20.99
2,1,14714,1,ACADEMY DINOSAUR,86,1,6,20.99
3,1,16050,1,ACADEMY DINOSAUR,86,1,6,20.99
4,2,972,1,ACADEMY DINOSAUR,86,1,6,20.99


In [68]:
#Split data

categorical = df.select_dtypes(include = object)
numerical = df.select_dtypes(exclude = object) 

In [69]:
numerical

,inventory_id,rental_id,film_id,length,store_id,rental_duration,replacement_cost
0,1,4863,1,86,1,6,20.99
1,1,11433,1,86,1,6,20.99
2,1,14714,1,86,1,6,20.99
3,1,16050,1,86,1,6,20.99
4,2,972,1,86,1,6,20.99
...,...,...,...,...,...,...,...
16040,4581,711,1000,50,2,3,18.99
16041,4581,1493,1000,50,2,3,18.99
16042,4581,6712,1000,50,2,3,18.99
16043,4581,9701,1000,50,2,3,18.99


In [70]:
categorical

,title
0,ACADEMY DINOSAUR
1,ACADEMY DINOSAUR
2,ACADEMY DINOSAUR
3,ACADEMY DINOSAUR
4,ACADEMY DINOSAUR
...,...
16040,ZORRO ARK
16041,ZORRO ARK
16042,ZORRO ARK
16043,ZORRO ARK


In [71]:
#Create X to apply the model latter
X = numerical 

Create a query to get the list of films and a boolean indicating if it was rented last month. 
This would be our target variable.

In [25]:
#cursor = db.cursor()
#query = "select * from (select film_id, title from sakila.film)sub1 cross join (select rental_date from sakila.rental where rental_date like '____-05-%')sub2"
#cursor.execute(query)


In [26]:
#df2 = pd.DataFrame(cursor.fetchall())
#df2

,0,1,2
0,1000,ZORRO ARK,2005-05-24 22:53:30
1,999,ZOOLANDER FICTION,2005-05-24 22:53:30
2,998,ZHIVAGO CORE,2005-05-24 22:53:30
3,997,YOUTH KICK,2005-05-24 22:53:30
4,996,YOUNG LANGUAGE,2005-05-24 22:53:30
...,...,...,...
1155995,5,AFRICAN EGG,2005-05-31 22:47:45
1155996,4,AFFAIR PREJUDICE,2005-05-31 22:47:45
1155997,3,ADAPTATION HOLES,2005-05-31 22:47:45
1155998,2,ACE GOLDFINGER,2005-05-31 22:47:45


In [ ]:
# Comment:
#With this query I've got more rows than with the previous one, It does not help me to develop the model later since there would be a lot of imbalance in the data.
#Instead I have decided to use the first query importing less features


In [72]:
cursor = db.cursor() #film_id, title, rental_date
query = "select l.rental_date, a.film_id, a.title from rental l join inventory i on l.inventory_id = i.inventory_id join film a on i.film_id = a.film_id;"
cursor.execute(query)


In [73]:
df3 = pd.DataFrame(cursor.fetchall())
df3

,0,1,2
0,2005-07-08 19:03:15,1,ACADEMY DINOSAUR
1,2005-08-02 20:13:10,1,ACADEMY DINOSAUR
2,2005-08-21 21:27:43,1,ACADEMY DINOSAUR
3,2022-08-26 16:54:15,1,ACADEMY DINOSAUR
4,2005-05-30 20:21:07,1,ACADEMY DINOSAUR
...,...,...,...
16040,2005-05-29 03:49:03,1000,ZORRO ARK
16041,2005-06-15 21:50:32,1000,ZORRO ARK
16042,2005-07-12 13:24:47,1000,ZORRO ARK
16043,2005-07-31 07:32:21,1000,ZORRO ARK


In [74]:
col_names3 = ['rented_last_month','film_id', 'title']
df3.columns = col_names3
df3.columns

Index(['rented_last_month', 'film_id', 'title'], dtype='object')

In [75]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16045 entries, 0 to 16044
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   rented_last_month  16045 non-null  datetime64[ns]
 1   film_id            16045 non-null  int64         
 2   title              16045 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 376.2+ KB


In [76]:
df3["rented_last_month"] = df3.rented_last_month.dt.strftime('%Y-%m-%d')

In [77]:
df3["rented_last_month"].value_counts()

2005-07-31    679
2005-08-01    671
2005-08-21    659
2005-07-27    649
2005-08-02    643
2005-07-29    641
2005-07-30    634
2005-08-19    628
2005-08-22    626
2005-08-20    624
2005-08-18    621
2005-07-28    620
2005-08-23    598
2005-08-17    593
2005-07-09    513
2005-07-08    512
2005-07-06    504
2005-07-12    495
2005-07-10    480
2005-07-07    461
2005-07-11    461
2005-06-15    348
2005-06-19    348
2005-06-18    344
2005-06-20    331
2005-06-17    325
2005-06-16    324
2005-06-21    275
2005-05-28    196
2006-02-14    182
2005-05-26    174
2005-05-27    166
2005-05-31    163
2005-05-30    158
2005-05-29    154
2005-05-25    137
2005-07-26     33
2005-07-05     27
2005-08-16     23
2005-06-14     16
2005-05-24      8
2022-08-26      1
Name: rented_last_month, dtype: int64

In [ ]:
#Although SQL queries showed that last month was May, looking at this data it seems that last month is 08-2005. 
#I have an entry from 08-2022 but as is only one and is not enough data I'll use data from 08-2005 as 1s.

In [78]:
last_month = [
            (df3['rented_last_month'] >= '2005-08-%'),
            (df3['rented_last_month'] <= '2005-08-%')
]
             
choicelist = [1,0]

df3['rented_last_month']=np.select(last_month, choicelist)
df3.head()

#Doc used : https://www.youtube.com/watch?v=h2yDbU1VDsI

,rented_last_month,film_id,title
0,0,1,ACADEMY DINOSAUR
1,1,1,ACADEMY DINOSAUR
2,1,1,ACADEMY DINOSAUR
3,1,1,ACADEMY DINOSAUR
4,0,1,ACADEMY DINOSAUR


In [79]:
df3['rented_last_month'].value_counts() #1 is films rented last month, 0 is films rented before

0    10176
1     5869
Name: rented_last_month, dtype: int64

In [80]:
#defining target variable

y = df3['rented_last_month']

In [ ]:
#Create a logistic regression model to predict this variable from the cleaned data

In [81]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.compose import ColumnTransformer

from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

In [82]:
X

,inventory_id,rental_id,film_id,length,store_id,rental_duration,replacement_cost
0,1,4863,1,86,1,6,20.99
1,1,11433,1,86,1,6,20.99
2,1,14714,1,86,1,6,20.99
3,1,16050,1,86,1,6,20.99
4,2,972,1,86,1,6,20.99
...,...,...,...,...,...,...,...
16040,4581,711,1000,50,2,3,18.99
16041,4581,1493,1000,50,2,3,18.99
16042,4581,6712,1000,50,2,3,18.99
16043,4581,9701,1000,50,2,3,18.99


In [83]:
y

0        0
1        1
2        1
3        1
4        0
        ..
16040    0
16041    0
16042    0
16043    0
16044    1
Name: rented_last_month, Length: 16045, dtype: int64

In [ ]:
#Comment: not sure why but X has lower number of samples than y. I will sample y to match the number of samples of X

In [84]:
y = y.sample(15862)

In [85]:
y.shape

(15862,)

In [86]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 1)

In [87]:
preprocessor = StandardScaler()
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

In [88]:
classifier = LogisticRegression()
classifier.fit(X_train, y_train)

LogisticRegression()

In [89]:
print("Train Score : {}".format(classifier.score(X_train, y_train)))
print('Test Score : {}'.format(classifier.score(X_test, y_test)))

Train Score : 0.6346933261280735
Test Score : 0.6341668417734818


In [90]:
#Predictions on train and test
pred_train = classifier.predict(X_train)
pred_test = classifier.predict(X_test)

In [91]:
#Evaluate the results
print(classification_report(y_train,pred_train))
print(classification_report(y_test, pred_test))

              precision    recall  f1-score   support

           0       0.63      1.00      0.78      7047
           1       0.00      0.00      0.00      4056

    accuracy                           0.63     11103
   macro avg       0.32      0.50      0.39     11103
weighted avg       0.40      0.63      0.49     11103

              precision    recall  f1-score   support

           0       0.63      1.00      0.78      3018
           1       0.00      0.00      0.00      1741

    accuracy                           0.63      4759
   macro avg       0.32      0.50      0.39      4759
weighted avg       0.40      0.63      0.49      4759



/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.9/site-packages/sklearn/metr

#Comment:
** Precision, recall and F1_score for the 1 (films rented last month) is 0 in all the three metrics.
** Precision: Of all the films the model predicted would be rented (0.63), 0 actually did.
** Recall: Of all the films that did get rented (1.00), the model couldn't predict any (0).
** F1_score: it's 0.00 to predict the films that will be rented, so the model does a poor job on predicting wheter a film will be rented or not.

** The model is too simple (underfitting):can't predict films that will be rented, also it's not learning any patterns of the data
